In [1]:
import pandas as pd
import numpy as np
concrete_data = pd.read_csv('https://raw.githubusercontent.com/hsgeral/IBM-AI-Engineering-Professional-Certificate/main/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [2]:
concrete_data.shape

(1030, 9)

## Split data into predictors and target¶

In [3]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [4]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [5]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Normalize the data by substracting the mean and dividing by the standard deviation.

In [6]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()

In [7]:
n_cols = predictors_norm.shape[1] # number of predictors

## Start the Neural Network part

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Defining a function that creates a NN with 
- Three hidden layers of num_nodes, and a ReLU activation function
- Use the adam optimizer and the mean squared error  as the loss function.

In [9]:
# define regression model
def regression_model_3hidden(num_nodes, num_inputs, num_outputs):
    # create model
    model = Sequential()
    model.add(Dense(num_nodes, activation='relu', input_shape=(num_inputs,)))
    model.add(Dense(num_nodes, activation='relu'))
    model.add(Dense(num_nodes, activation='relu'))
    model.add(Dense(num_outputs))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Train and Test the Network
- using the fit method to train and test the model at the same time,
- leaving out 30% of the data for validation 
- using 50 epochs to train the model.
- repeating 50 times to collect the 50 mean squared errors.

The model will set apart the 'validation_split' fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 

In [ ]:
num_nodes = 10
num_inputs = n_cols
num_outputs = 1
mse_list = [] # Initialize an empty list to store mean squared errors
for i in range(50):
    model = regression_model_3hidden(num_nodes, num_inputs, num_outputs) # build the model
    history=model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=0,shuffle=True) # fit the model
    mse_list.append(history.history['loss'][-1]) # get the mean square error after the 50th epoch
    print(i,mse_list[i])
# in each of the 50 repetitions the model was recreated otherwise the loss would constinously decrease as we were 
# fitting 50 * 50 epochs, increasing the number of epochs would result in much lower losses, but the exercised required to 
# limit it to 50
          
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)
print (mean_mse,std_mse)

2024-06-20 17:40:39.789989: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2024-06-20 17:40:39.794866: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394310000 Hz
2024-06-20 17:40:39.795473: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x563587f6c7c0 executing computations on platform Host. Devices:
2024-06-20 17:40:39.795520: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2024-06-20 17:40:39.919671: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

0 135.09791143790363
1 142.51240837954285
2 143.8671502101438
3 119.26786592565529
4 147.44914939763973
5 107.5647660465743
6 94.26824254897035
7 95.75190008959724
8 130.24035708021358
9 133.03861385774016
10 99.68201407380971
11 98.8841284277039
12 131.9641878971941
13 137.52815324886495
14 129.12050414912082
15 143.4829105054456
16 135.05565732196698
17 137.62001780431908
18 82.40275328235388
19 144.66052401644512
20 143.05917545694254
21 142.50908083128698
22 142.12634220202654
23 119.90047689747381
24 140.4389289930029
25 138.86198637984828
26 138.2292322801651
27 148.86431461498245
28 138.9832406328389
29 143.33399885074442
30 133.92912467078264
31 136.7866368075515
32 144.92232089102185
33 139.55716996318588
34 135.59818704845836
35 146.5732183575465
36 144.43985525017473
37 136.26338437063188
38 127.67877598311468
39 140.1552126351408
40 109.68280226116207
41 132.0951323277742
42 142.5621639151183
43 144.73081667578666
44 134.0920208680977
45 146.05041177990367
46 131.9020424894